# Imports and Utility Functions

In [25]:
import codecs


def repeat_key_xor(message, key):
    n, m = len(message), len(key)
    result = []
    for i in range(n):
        byte = message[i] ^ key[i % m]
        result.append(byte)
    return bytes(result)


LETTERS = 'etaoinsrhldcumfpgwybvkxjqz'
SCORES = [x for x in range(len(LETTERS), 0, -1)]
def text_score(S):
    """
    Returns the score of a piece of text looking only at letter 
    frequency.
    """
    score = 0
    for i in S:
        ch = chr(i)
        if ch != '\n' and not ch.isprintable():
            return -1
        if ch in LETTERS:
            score += SCORES[LETTERS.index(ch)]
    return score


def popcount(x):
    return bin(x).count('1')


def hamming_distance(A, B):
    """
    (bytes, bytes) -> int
    Returns the hamming distance between 'bytes' A and B.
    """
    assert len(A) == len(B), 'lengths are not equal.'

    distance = sum(popcount(x ^ y) for x, y in zip(A, B))
    return distance


## Challenge 1.1
Convert hex to base64

In [26]:
hex_string = '49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d'
b = codecs.decode(hex_string, 'hex')
base64_string = codecs.encode(b, 'base64')

base64_string

b'SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t\n'

## Challenge 1.2: Fixed XOR
Write a function that takes two equal-length buffers and produces their XOR combination.

In [27]:
hex1 = '1c0111001f010100061a024b53535009181c'
hex2 = '686974207468652062756c6c277320657965'
b1 = codecs.decode(hex1, 'hex')
b2 = codecs.decode(hex2, 'hex')

result = [x ^ y for x, y in zip(b1, b2)]
bytes(result).hex()

'746865206b696420646f6e277420706c6179'

## Challenge 1.3: Single-byte XOR cipher

In [28]:
def single_key_xor(b):
    best = (-1, b'')
    for guess in range(256):
        b1 = repeat_key_xor(b, bytes([guess]))
        score = text_score(b1)
        best = max(best, (score, b1))
    return best

hex1 = '1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736'
b1 = codecs.decode(hex1, 'hex')
    
single_key_xor(b1)

(428, b"Cooking MC's like a pound of bacon")

## Challenge 1.4: Detect single-character XOR

In [35]:
best = (-1, b'')
with open('data/4.txt', 'r') as fin:
    for line in fin:
        ans = single_key_xor(codecs.decode(line.rstrip(), 'hex'))
        best = max(best, ans)
best

(416, b'Now that the party is jumping\n')

## Challenge 1.5: Implement repeating-key XOR

In [30]:
msg = '''Burning 'em, if you ain't quick and nimble
I go crazy when I hear a cymbal'''

key = 'ICE'
b = repeat_key_xor(bytes(ord(x) for x in msg), bytes(ord(x) for x in key))

codecs.encode(b, 'hex')

b'0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63343c2a26226324272765272a282b2f20430a652e2c652a3124333a653e2b2027630c692b20283165286326302e27282f'

## Challenge 1.6: Break repeating-key XOR

In [31]:
s1 = 'this is a test'
s2 = 'wokka wokka!!!'

hamming_distance(s1.encode('utf-8'), s2.encode('utf-8'))

37

In [34]:
msg = codecs.decode(open('data/6.txt', 'r').read().encode('utf-8'), 'base64')
n = len(msg)
keysizes = []
for k in range(2, 41):
    total_distance = 0
    for i in range(0, n, k):
        if i + 2*k >= n:
            break
        total_distance += hamming_distance(msg[i:i + k], msg[i + k:i + 2*k])
    keysizes.append((k, total_distance))

keysizes.sort(key=lambda x: x[1])
for k in keysizes[:1]:
    size = k[0]
    blocks = [[] for _ in range(size)]
    for i in range(n):
        blocks[i % size].append(msg[i])

    best = []
    for block in blocks:
        best.append(single_key_xor(bytes(block))[1])

    reconstructed = []
    for i in range(n):
        reconstructed.append(best[i % size][i // size])
        
    print(bytes(reconstructed).decode('utf-8'))


I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you can bet 
I can take a fly girl and make her wet. 


## Challenge 1.7: AES in ECB mode

In [36]:
def flip_16(b):
    result = [0 for _ in range(16)]
    for i in range(16):
        r, c = i // 4, i % 4
        result[4*c + r] = b[i]
    return result


sbox = [
    0x63, 0x7C, 0x77, 0x7B, 0xF2, 0x6B, 0x6F, 0xC5, 0x30, 0x01, 0x67, 0x2B,
    0xFE, 0xD7, 0xAB, 0x76, 0xCA, 0x82, 0xC9, 0x7D, 0xFA, 0x59, 0x47, 0xF0,
    0xAD, 0xD4, 0xA2, 0xAF, 0x9C, 0xA4, 0x72, 0xC0, 0xB7, 0xFD, 0x93, 0x26,
    0x36, 0x3F, 0xF7, 0xCC, 0x34, 0xA5, 0xE5, 0xF1, 0x71, 0xD8, 0x31, 0x15,
    0x04, 0xC7, 0x23, 0xC3, 0x18, 0x96, 0x05, 0x9A, 0x07, 0x12, 0x80, 0xE2,
    0xEB, 0x27, 0xB2, 0x75, 0x09, 0x83, 0x2C, 0x1A, 0x1B, 0x6E, 0x5A, 0xA0,
    0x52, 0x3B, 0xD6, 0xB3, 0x29, 0xE3, 0x2F, 0x84, 0x53, 0xD1, 0x00, 0xED,
    0x20, 0xFC, 0xB1, 0x5B, 0x6A, 0xCB, 0xBE, 0x39, 0x4A, 0x4C, 0x58, 0xCF,
    0xD0, 0xEF, 0xAA, 0xFB, 0x43, 0x4D, 0x33, 0x85, 0x45, 0xF9, 0x02, 0x7F,
    0x50, 0x3C, 0x9F, 0xA8, 0x51, 0xA3, 0x40, 0x8F, 0x92, 0x9D, 0x38, 0xF5,
    0xBC, 0xB6, 0xDA, 0x21, 0x10, 0xFF, 0xF3, 0xD2, 0xCD, 0x0C, 0x13, 0xEC,
    0x5F, 0x97, 0x44, 0x17, 0xC4, 0xA7, 0x7E, 0x3D, 0x64, 0x5D, 0x19, 0x73,
    0x60, 0x81, 0x4F, 0xDC, 0x22, 0x2A, 0x90, 0x88, 0x46, 0xEE, 0xB8, 0x14,
    0xDE, 0x5E, 0x0B, 0xDB, 0xE0, 0x32, 0x3A, 0x0A, 0x49, 0x06, 0x24, 0x5C,
    0xC2, 0xD3, 0xAC, 0x62, 0x91, 0x95, 0xE4, 0x79, 0xE7, 0xC8, 0x37, 0x6D,
    0x8D, 0xD5, 0x4E, 0xA9, 0x6C, 0x56, 0xF4, 0xEA, 0x65, 0x7A, 0xAE, 0x08,
    0xBA, 0x78, 0x25, 0x2E, 0x1C, 0xA6, 0xB4, 0xC6, 0xE8, 0xDD, 0x74, 0x1F,
    0x4B, 0xBD, 0x8B, 0x8A, 0x70, 0x3E, 0xB5, 0x66, 0x48, 0x03, 0xF6, 0x0E,
    0x61, 0x35, 0x57, 0xB9, 0x86, 0xC1, 0x1D, 0x9E, 0xE1, 0xF8, 0x98, 0x11,
    0x69, 0xD9, 0x8E, 0x94, 0x9B, 0x1E, 0x87, 0xE9, 0xCE, 0x55, 0x28, 0xDF,
    0x8C, 0xA1, 0x89, 0x0D, 0xBF, 0xE6, 0x42, 0x68, 0x41, 0x99, 0x2D, 0x0F,
    0xB0, 0x54, 0xBB, 0x16
]

inv_sbox = [
    0x52, 0x09, 0x6A, 0xD5, 0x30, 0x36, 0xA5, 0x38, 0xBF, 0x40, 0xA3, 0x9E,
    0x81, 0xF3, 0xD7, 0xFB, 0x7C, 0xE3, 0x39, 0x82, 0x9B, 0x2F, 0xFF, 0x87,
    0x34, 0x8E, 0x43, 0x44, 0xC4, 0xDE, 0xE9, 0xCB, 0x54, 0x7B, 0x94, 0x32,
    0xA6, 0xC2, 0x23, 0x3D, 0xEE, 0x4C, 0x95, 0x0B, 0x42, 0xFA, 0xC3, 0x4E,
    0x08, 0x2E, 0xA1, 0x66, 0x28, 0xD9, 0x24, 0xB2, 0x76, 0x5B, 0xA2, 0x49,
    0x6D, 0x8B, 0xD1, 0x25, 0x72, 0xF8, 0xF6, 0x64, 0x86, 0x68, 0x98, 0x16,
    0xD4, 0xA4, 0x5C, 0xCC, 0x5D, 0x65, 0xB6, 0x92, 0x6C, 0x70, 0x48, 0x50,
    0xFD, 0xED, 0xB9, 0xDA, 0x5E, 0x15, 0x46, 0x57, 0xA7, 0x8D, 0x9D, 0x84,
    0x90, 0xD8, 0xAB, 0x00, 0x8C, 0xBC, 0xD3, 0x0A, 0xF7, 0xE4, 0x58, 0x05,
    0xB8, 0xB3, 0x45, 0x06, 0xD0, 0x2C, 0x1E, 0x8F, 0xCA, 0x3F, 0x0F, 0x02,
    0xC1, 0xAF, 0xBD, 0x03, 0x01, 0x13, 0x8A, 0x6B, 0x3A, 0x91, 0x11, 0x41,
    0x4F, 0x67, 0xDC, 0xEA, 0x97, 0xF2, 0xCF, 0xCE, 0xF0, 0xB4, 0xE6, 0x73,
    0x96, 0xAC, 0x74, 0x22, 0xE7, 0xAD, 0x35, 0x85, 0xE2, 0xF9, 0x37, 0xE8,
    0x1C, 0x75, 0xDF, 0x6E, 0x47, 0xF1, 0x1A, 0x71, 0x1D, 0x29, 0xC5, 0x89,
    0x6F, 0xB7, 0x62, 0x0E, 0xAA, 0x18, 0xBE, 0x1B, 0xFC, 0x56, 0x3E, 0x4B,
    0xC6, 0xD2, 0x79, 0x20, 0x9A, 0xDB, 0xC0, 0xFE, 0x78, 0xCD, 0x5A, 0xF4,
    0x1F, 0xDD, 0xA8, 0x33, 0x88, 0x07, 0xC7, 0x31, 0xB1, 0x12, 0x10, 0x59,
    0x27, 0x80, 0xEC, 0x5F, 0x60, 0x51, 0x7F, 0xA9, 0x19, 0xB5, 0x4A, 0x0D,
    0x2D, 0xE5, 0x7A, 0x9F, 0x93, 0xC9, 0x9C, 0xEF, 0xA0, 0xE0, 0x3B, 0x4D,
    0xAE, 0x2A, 0xF5, 0xB0, 0xC8, 0xEB, 0xBB, 0x3C, 0x83, 0x53, 0x99, 0x61,
    0x17, 0x2B, 0x04, 0x7E, 0xBA, 0x77, 0xD6, 0x26, 0xE1, 0x69, 0x14, 0x63,
    0x55, 0x21, 0x0C, 0x7D
]


def key_expansion(key):
    rcon = [1, 2, 4, 8, 16, 32, 64, 128, 27, 54, 108, 216, 171]
    rcon_i = 0
    expanded_key = [list(key)]
    for rd in range(1, 11):
        current_key = []
        for k in range(4):
            t = current_key[-4:] if k > 0 else expanded_key[rd-1][-4:]
            if k == 0:
                t = t[1:] + t[:1]
                for i in range(0, 4):
                    t[i] = sbox[t[i]]
                t[0] ^= rcon[rcon_i]
                rcon_i += 1
            for i in range(0, 4):
                t[i] ^= expanded_key[rd-1][k*4 + i]
            current_key += t
        expanded_key.append(current_key)
    return expanded_key


def sub_bytes(state):
    return [sbox[x] for x in state]


def inv_sub_bytes(state):
    return [inv_sbox[x] for x in state]


def shift_rows(state):
    state = flip_16(state)
    result = []
    for i in range(4):
        a, b = 4 * i, 4 * (i+1)
        result += state[a+i:b] + state[a:a+i]
    result = flip_16(result)
    return result


def inv_shift_rows(state):
    state = flip_16(state)
    result = []
    for i in range(4):
        a, b = 4 * i, 4 * (i+1)
        result += state[a+(4-i):b] + state[a:a+(4-i)]
    result = flip_16(result)
    return result


def gmul(a, b):
    p = 0
    for _ in range(8):
        if (b & 1) == 1:
            p ^= a
        high = (a & (1 << 7)) != 0
        a <<= 1
        if high:
            a ^= int('1b', 16)
        b >>= 1
    return p % 256


def mix_column(column):
    mat = [
        [2, 3, 1, 1],
        [1, 2, 3, 1],
        [1, 1, 2, 3],
        [3, 1, 1, 2]]
    result = []
    for i in range(4):
        x = 0
        for j in range(4):
            x ^= gmul(mat[i][j], column[j])
        result.append(x)
    return result


def mix_columns(state):
    for i in range(4):
        column = state[4*i:4*(i+1)]
        column = mix_column(column)
        state[4*i:4*(i+1)] = column
    return state


def inv_mix_column(column):
    mat = [
        [0x0e, 0x0b, 0x0d, 0x09],
        [0x09, 0x0e, 0x0b, 0x0d],
        [0x0d, 0x09, 0x0e, 0x0b],
        [0x0b, 0x0d, 0x09, 0x0e]]
    result = []
    for i in range(4):
        x = 0
        for j in range(4):
            x ^= gmul(mat[i][j], column[j])
        result.append(x)
    return result


def inv_mix_columns(state):
    for i in range(4):
        column = state[4*i:4*(i+1)]
        column = inv_mix_column(column)
        state[4*i:4*(i+1)] = column
    return state


def add_round_key(state, key):
    for i in range(16):
        state[i] ^= key[i]
    return state


def aes_128_encrypt(text, key):
    assert len(text) == 16
    assert len(key) == 16
    block = list(text)
    round_key = key_expansion(key)
    block = add_round_key(block, round_key[0])
    for rd in range(10):
        block = sub_bytes(block)
        block = shift_rows(block)
        if rd < 9:
            block = mix_columns(block)
        block = add_round_key(block, round_key[rd+1])
    return bytes(block)


def aes_128_decrypt(cipher, key):
    assert len(cipher) == 16
    assert len(key) == 16
    block = list(cipher)
    round_key = key_expansion(key)
    for rd in range(10):
        block = add_round_key(block, round_key[10 - rd])
        if rd > 0:
            block = inv_mix_columns(block)
        block = inv_sub_bytes(block)
        block = inv_shift_rows(block)
    block = add_round_key(block, round_key[0])
    return bytes(block)

In [39]:
msg = codecs.decode(open('data/7.txt', 'r').read().encode('utf-8'), 'base64')
key = b'YELLOW SUBMARINE'

n = len(msg)
text = b''
for k in range(0, n, 16):
    cipher = msg[k:(k+16) if (k+16) < n else n]
    if len(cipher) != 16:
        # TODO: figure out padding here
        continue
    text += aes_128_decrypt(cipher, key)
print(text.decode('utf-8'))

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you can bet 
I can take a fly girl and make her wet. 


## Challenge 1.8: Detect AES in ECB mode

In [58]:
lines = open('data/8.txt', 'r').readlines()
for line in lines:
    b = codecs.decode(line.rstrip(), 'hex')
    n = len(b)
    
    S = set()
    for i in range(0, n, 16):
        S.add(b[i:i+16])
    
    if len(S) != (n // 16):
        print(line)

d880619740a8a19b7840a8a31c810a3d08649af70dc06f4fd5d2d69c744cd283e2dd052f6b641dbf9d11b0348542bb5708649af70dc06f4fd5d2d69c744cd2839475c9dfdbc1d46597949d9c7e82bf5a08649af70dc06f4fd5d2d69c744cd28397a93eab8d6aecd566489154789a6b0308649af70dc06f4fd5d2d69c744cd283d403180c98c8f6db1f2a3f9c4040deb0ab51b29933f2c123c58386b06fba186a

